## NLP (Natural Language Processing)

Entender todo lo relacionado con el lenguaje humano. Ejemplos:

- Clasificación de texto (e.g. predicción de spam).
- Generación de texto: Dada una entrada genera una salida.
- Convertir audios a texto y viceversa.
- Convertir un texto a una imagen y viceversa.

## Word embeddings

Los *word embeddings* son representaciones de texto, convirtiendo cada palabra a numérico (vector).

Estos word embeddings son el input para los diferentes modelos de ML.

Cada palabra en una frase es un *token*, que se convierte a un word embedding (0 1 0 1).

Se tiene una tabla que mapea cada palabra del voculario a la posición de su word embedding. 

Tenemos una gran matriz en la que cada palabra corresponde a una fila, y cada columna a una secuencia de números (word embedding).

El carácter UNK se utiliza para palabras fuera del vocabulario (*out-of-vocabulary*). 


#### One-hot encoding

Para la palabra *i* del vocabulario, el vector word embedding tiene valor 1 en la posición *i*, y 0 en el resto de posiciones.

Ejemplo: Vocabulario con tres palabras, siendo *Hola* la primera palabra. Su vector one-hot encoded es [1,0,0].

Problemas: 

- **Lack of scalability**: Normalmente no se tienen tres palabras sino que se tienen muchas más, dando pie a vectores enormes. 
- **Significado**: Se pierde la información sobre la relación entre las palabras, sobre su contexto en cada frase.

#### Word2vec

Modelo de ML en el que los parámetros son los vectores de cada palabra. Tras el entrenamiento, este modelo es capaz de mapear palabras a un espacio dimensional en el que los word embeddings mantienen información interesante sobre el contexto.

**Sliding window**: Una ventana abarca una serie de palabras contexto entorno a una palabra clave. Ejemplo: Hoy <span style='color:red'>creo que</span> <span style='color:green'>es</span> <span style='color:red'>un gran</span> día (ventana de 5 palabras en torno a *es*).

Variantes de word2vec:

- Skip-Gram: Predice el contexto dada una palabra central.

- Continuous bag of words (CBOW): Predice la palabra central sumando los vectores de contexto.

<img src="skip_gram_cbow.png">

#### Propiedades de los word embeddings

- Las relaciones semánticas y sintácticas son **lineares** en el espacio vectorial.
    - V(rey) - V(hombre) + V(mujer) = V(reina).
    - V(España) - V(Madrid) + V(Paris) = V(Francia).
- **Sesgos**: Al coger nuestra base de datos para entrenar el word2vec, la información puede estar sesgada. Por ejemplo, a un hombre le asigna la profesión "médico", mientras que a una mujer le asigna la profesión "enfermera", porque en la base de datos se aprecia este sesgo. Artículos sobre *Fairness* en NLP.
    

## Words and vectors

**Term-document matrix**: Cada fila representa una palabra en el vocabulario y cada columna representa un documento dentro de una colección. Cada celda representa el número de veces que una palabra aparece en un documento en particular. En esta tabla, cada columna representa un **vector** en el espacio. Estos vectores pueden ser utiliados para encontrar documentos similares entre sí. De igual manera, utilizando los vectores fila en lugar de los vectores columna, podemos obtener información sobre el significado de una palabra dados los documentos en los que tiende a aparecer.

**Information retrieval**: Tarea de encontrar un documento *d* en una colleción *D* dada una query *q*.

**Term-term matrix**: En esta matriz las columnas son palabras en lugar de documentos. Cada celda guarda el número de veces que una palabra fila (*target*) y una palabra columna (*contexto*) co-ocurren en un contexto dado un training corpus. El contexto suele ser una *sliding window* de en torno a 4 palabras. Esta matriz nos permite obtener la similitud entre diferentes palabras.

## Cosine similarity

Cosine similarity es la métrica más común para medir similitud entre word vectors:

$\cos(\vec{v},\vec{w}) = \frac{\vec{v}\cdot\vec{w}}{|\vec{v}||\vec{w}|}$

- $\cos(\vec{v},\vec{w}) = 1$: Los dos vectores apuntan en la misma dirección.
- $\cos(\vec{v},\vec{w}) = 0$: Los dos vectores apuntan en direcciones opuestas.

No es posible obtener valores negativos pues los vectores de frecuencia no tienen entradas negativas. Así pues, $\cos(\vec{v},\vec{w})~\epsilon~[0,1]$

## TF-IDF: Weighting terms in the vector

Contar la frecuencia de palabras no es la mejor manera de medir una asociación entre vectores. Esto es así dado que palabras omnipresentes como *el* o *bien* tendrían una frecuencia muy alta, mientras que la información que aportan es muy baja. Para solucionar este problema, se utiliza el TF-IDF weighting.

Esta técnica consiste en el producto de dos cantidades: 

- **Term frequency (tf)**: La frecuencia de la palabra *t* en un documento *d*.

$tf_{t,d} = \log_{10}(count(t,d)+1)$ 

- **Inverse document frequency (idf)**: Viene dado por el ratio $N/df_{t}$, donde $N$ is el número total de documentos de la colección y $df_{t}$ es el número de documentos en los cuales la palabra $t$ aparece.

$idf_{t} = \log_{10}\left(\frac{N}{df_t}\right)$

El **peso tf-idf** $w_{t,d}$ viene dado por $w_{t,d} = tf_{t,d} \times idf_{t}$. Este repesado se aplica sobre la *term-document matrix* para realizar *information retrieval*, pero también tiene un rol muy importante en otros aspectos de NLP. Constituye un gran punto de partida para diferentes tareas.

Con esta definición, palabras frecuentes en todos los documentos como *bien* tienen un peso muy bajo, dado el término **idf**.

## Positive Pointwise Mutual Information (PPMI)

PPMI se utiliza sobre la *term-term matrix* para medir la diferencia entre la frecuencia en que dos palabras co-ocurren en un texto con respecto a lo que a priori esperaríamos. 

$\text{PPMI}(w,c) = \text{max}(\log_{2}\frac{P(w,c)}{P(w)P(c)},0)$

Donde el numerador nos dice cómo de frecuente es encontrar dos palabras juntas ($w$ target, $c$ contexto) y el denominador nos dice la frecuencia con la que esperaríamos encontrar las dos palabras juntas asumiendo que son independientes:

$P(w,c) = \frac{f_{ij}}{\sum_{i=1}^{W}\sum_{j=1}^{C}f_{ij}}$, $P(w) = \frac{\sum_{j=1}^{C}f_{ij}}{\sum_{i=1}^{W}\sum_{j=1}^{C}f_{ij}}$, $P(c) = \frac{\sum_{i=1}^{W}f_{ij}}{\sum_{i=1}^{W}\sum_{j=1}^{C}f_{ij}}$

Siendo $f_{ij}$ el elemento asociado a la *term-term matrix*.

En general, un problema de PPMI es que tiende a mostrar un bias hacia palabras poco frecuentes, a las cuales se les asocia valores muy altos del PPMI. Para resolver este problema, existen varias soluciones, como cambiar la función $P(c)$ incluyendo un exponente $\alpha \sim 0.75$, o introduciendo la técnica de *Laplace smoothing*, que consiste en introducir una pequeña constante $k$ a cada una de las cuentas con el fin de reducir el peso de las cuentas próximas a cero.

## Aplicaciones de TF-IDF y PPMI

El modelo empleado para estimar la similitud entre dos palabras o dos documentos consiste en aplicar la función **cosine similarity** entre los vectores **tf-idf** o **PPMI** de los dos objetos asociados. 

En general, **tf-idf** se utiliza para computar la similitud entre dos documentos. Para representar un documento, se toman los vectores tf-idf de todas las palabras en el documento y se calcula el centroide de todos esos vectores. Esto puede entenderse como una media multidimensional:

$d = \frac{w_1 + w_2 + ... + w_k}{k}$

Siendo $k$ el número de palabras en el documento y $w_i$ los vectores tf-idf asociados a cada palabra. 

Finalmente, la similitud entre dos documentos se calcula como $\cos(d_1,d_2)$.

Tanto PPMI como tf-idf se puden emplear para computar la similitud entre palabras, siguiendo un procedimiento similar.

## Word2vec: Expansión

La principal diferencia entre un vector palabra y un word embedding es que un vector de palabra es muy largo, siendo su longitud el número total de palabras en el vocabulario (*term-term matrix*) o el número total de documentos (*term-document matrix*). En cambio, los word embeddings son mucho más cortos y mucho más densos, lo cual mejora el rendimiento de los clasificadores y reduce las posibilidades de overfitting.

Uno de los métodos más utilizados para calcular los word embeddings es el **skip-gram with negative sampling (SGNS)**. El objetivo de este método es obtener un embedding estático (fijo) para cada palabra del vocabulario. Se pretender entrenar un clasificador binario que sea capaz de determinar la probabilidad de que una palabra target $w$ aparezca cerca de una palabra contexto *c*. Los pesos que aprende el clasificador en esta tarea se convierten en los word embeddings asociados a cada palabra.

Una ventaja de los word2vec respecto a las redes neuronales es que simplifican tanto la tarea (clasificación binaria es más sencilla que predicción de palabras) y también la arquitectura (es más sencillo entrenar un modelo de regresión logística que una red neuronal). El SGNS se puede resumir en los siguientes pasos:

1. Tratar la palabra target y las palabras contexto como ejemplos positivos.
2. Hacer un muestro aleatorio de las otras palabras del vocabulario para tener muestras negativas.
3. Utilizar regresión logística para entrenar el clasificador de cara a distinguir ambos casos.
4. Utilizar los pesos aprendidos como embeddings.

#### El clasificador

En primer lugar, el clasificador asigna un embedding aleatorio para cada palabra del vocabulario, y tiene como objetivo ir cambiando este embedding iterativamente para que sea más parecido al embedding de aquellas palabras que ocurren en contextos similares. 

Se calcula la probabilidad de que una palabra *c* sea parte del contexto de una palabra target *w* como:

$P(+|w,c) = \sigma(\vec{c}\cdot\vec{w}) = \frac{1}{1+\exp(-\vec{c}\cdot\vec{w})}$, donde $\sigma$ es la función sigmoide.

Para el conjunto de palabras en una ventana de contexto de tamaño L, tenemos:

$\log P(+|w,c_{1:L}) = \sum_{i=1}^{L}\log\sigma(\vec{c_{i}}\cdot\vec{w})$

La función de pérdida a minimizar por el clasificador viene dada por:

$\mathcal{L} = - \left[\log\sigma(c_{pos}\cdot{w}) + \sum_{i=1}^{k}\sigma(-c_{neg,i}\cdot{w})\right]$

Esta función tiene por objeto maximizar la similitud entre la palabra target y sus palabras contexto $c_{pos}$, extraídas a partir de los ejemplos positivos, y a la vez minimizar la similitud entre la palabra target y las palabras que no pertenecen al contexto $c_{neg}$, extraídas a partir de los muestreos negativos.

Para minimizar la función de pérdida, se utiliza el *stochastic gradient descent* algorithm. 

En conjunto, el algoritmo empieza con una inicialización aleatoria de las matrices $\boldsymbol{W}$ y $\boldsymbol{C}$, y realiza el entrenamiento utilizando *gradient descent* para modificar estas matrices con el fin de maximizar la función objetivo. Se acaban aprendiendo dos *embeddings* para cada palabra, el **target embedding** $\boldsymbol{w_{i}}$ y el **context embedding** $\boldsymbol{c
_{i}}$, guardados en la matriz target $\boldsymbol{W}$ y la matriz contexto $\boldsymbol{C}$.

In [1]:
## Gensim is an open-source library for unsupervised topic modelling and NLP,
## using modern statistical ML
!pip install --upgrade gensim

     |████████████████████████████████| 24.0 MB 30.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 14.5 MB/s eta 0:00:01


In [ ]:
# Load pretrained model on google news to perform simple actions with word embeddings
import gensim.downloader as api
model = api.load('word2vec-google-news-300')

In [4]:
#Obtain similarities from pre-trained model
sim = model.similarity("king", "queen")
dif = model.similarity("king","potato")
print (f"Model similarity between king and queen is {sim}.")
print (f"Model similarity between king and potato is {dif}.")

Model similarity between king and queen is 0.6510956883430481.
Model similarity between king and potato is 0.09978463500738144.


In [5]:
#Obtain top 5 similar words to a set of words
model.most_similar(["king", "queen"],topn=5)


[('monarch', 0.7042065858840942),
 ('kings', 0.6780862808227539),
 ('princess', 0.6731551885604858),
 ('queens', 0.6679496765136719),
 ('prince', 0.6435247659683228)]

In [6]:
# Check which word does not fit to a list
model.doesnt_match(["summer", "fall", "spring", "air"])

'air'

## Ejercicios

1. Usa el modelo word2vec para hacer un raking de las siguientes 15 palabras según su similitud con las palabras "man" y "woman". Para cada par, imprime su similitud.

In [1]:
words = [
"wife",
"husband",
"child",
"queen",
"king",
"man",
"woman",
"birth",
"doctor",
"nurse",
"teacher",
"professor",
"engineer",
"scientist",
"president"]


dic_man = {}
dic_woman = {}

for word in words:
    dic_man[word] = model.similarity("man",word)
    dic_woman[word] = model.similarity("woman",word)

sorted_man = {k: v for k, v in sorted(dic_man.items(), reverse=True, key=lambda item: item[1])}
sorted_woman = {k: v for k, v in sorted(dic_woman.items(), reverse=True, key=lambda item: item[1])}

print (list(sorted_man.keys()))
print (list(sorted_woman.keys()))
    

NameError: name 'model' is not defined

2. Completa las siguientes analogías

In [16]:
# man is to woman as king is to ...
model.most_similar(positive=["king", "woman"], negative=["man"], topn=1)

[('queen', 0.7118193507194519)]

In [17]:
#nurse is to hospital as teacher is to ...
model.most_similar(positive=["teacher", "hospital"], negative=["nurse"], topn=1)

[('school', 0.60170978307724)]